In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime
import os
from dotenv import load_dotenv

load_dotenv()

ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

from src.config.paths import *
from src.config.settings import *


In [2]:
# Some notes:
# We're missing too much datas for eurostoxx 50, so we will uses different stocks for now
# Dow Jones Global Titans 50 Index

# Ticker information for Dow Jones Global Titans 50 Index
# Bloomberg Ticker: DJGT
# Reuters: .DJGT
# Price Return: DJGT
# Total Return: DJGTG
# Net Total Return: DJGTT

DJ_TITANS_50_TICKER = [
    # "MMM",  # 3M
    "ABBV", # AbbVie
    "ALIZY", # Allianz #doesnt work
    "GOOG", # Alphabet
    "AMZN", # Amazon
    "AMGN", # Amgen
    "BUD",  # Anheuser-Busch InBev
    "AAPL", # Apple
    "BHP",  # BHP
    "BA",   # Boeing
    "BP",   # BP
    "BTI",  # British American Tobacco
    "CVX",  # Chevron
    "CSCO", # Cisco
    "C",    # Citigroup
    "KO",   # Coca-Cola
    "DD",   # DuPont
    "XOM",  # ExxonMobil
    "META", # Meta
    "GE",   # General Electric
    "GSK",  # GlaxoSmithKline
    "HSBC", # HSBC
    "INTC", # Intel
    "IBM",  # IBM
    "JNJ",  # Johnson & Johnson
    "JPM",  # JPMorgan Chase
    "MA",   # Mastercard
    "MCD",  # McDonald's
    "MRK",  # Merck
    "MSFT", # Microsoft
    "NSRGY", # Nestlé  #doesnt work
    "NVS",  # Novartis
    "NVDA", # Nvidia
    "ORCL", # Oracle
    "PEP",  # PepsiCo
    "PFE",  # Pfizer
    "PM",   # Philip Morris
    "PG",   # Procter & Gamble
    "RHHBY", # Roche  #doesnt work
    "RY",   # Royal Bank of Canada
    "SHEL", # Shell
    "SSNLF", # Samsung  #doesnt work
    "SNY",  # Sanofi
    "SIEGY", # Siemens  #doesnt work
    "TSM",  # TSMC
    "TTE",  # TotalEnergies
    "V",    # Visa
    "TM",   # Toyota
    "WMT",  # Walmart
    "DIS",   # Disney
    "DJGT"  # DJ Global Titans 50 Index  #doesnt work
]

In [24]:
import time
import logging
from datetime import datetime, timedelta
from src.utils.log_utils import setup_logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = setup_logging(__name__)

def fetch_market_intraday(ticker: str, interval: str = "1min", output_dir: Path = MARKET_DATA_CACHE, api_key: str = ALPHA_VANTAGE_API_KEY, datatype: str = "csv", adjusted: bool = True, extended_hours: bool = True, outputsize: str = "full"):
    """
    Fetch intraday market data for a given ticker from Alpha Vantage.
    For 1min interval, fetches data month by month for the past year.
    For other intervals, fetches the most recent 30 days of data.

    Args:
        ticker (str): The ticker symbol of the stock to fetch data for.
        interval (str): Time interval between data points. One of: 1min, 5min, 15min, 30min, 60min
        output_dir (Path): The directory to save the fetched data.
        api_key (str): Alpha Vantage API key
        datatype (str): Return data format - 'json' or 'csv'
        adjusted (bool): Whether to adjust for splits/dividends. Default True
        extended_hours (bool): Whether to include extended trading hours. Default True
        outputsize (str): Amount of data to return - 'compact' or 'full'. Default 'full'

    Returns:
        Path: Path to the saved data file
    """
    logger.info(f"Starting data fetch for {ticker} with {interval} interval")
    data_file = output_dir / f"{ticker}_{interval}.{datatype}"
    
    # For 1min interval, fetch month by month for past year
    if interval == "1min":
        logger.info("Using 1min interval - fetching data month by month for past year")
        # Get list of months to fetch (past 12 months)
        months = []
        current_date = datetime.now()
        for i in range(12):
            first_of_month = current_date.replace(day=1)
            months.append(first_of_month.strftime("%Y-%m"))
            current_date = first_of_month - timedelta(days=1)
        
        logger.info(f"Will fetch data for months: {months}")
        
        # Fetch data for each month
        all_data = []
        header = None
        for month in months:
            logger.info(f"Fetching data for {ticker} - {month}")
            params = {
                'function': 'TIME_SERIES_INTRADAY',
                'symbol': ticker,
                'interval': interval,
                'apikey': api_key,
                'datatype': datatype,
                'adjusted': str(adjusted).lower(),
                'extended_hours': str(extended_hours).lower(),
                'outputsize': outputsize,
                'month': month
            }
            
            url = 'https://www.alphavantage.co/query'
            r = requests.get(url, params=params)
            
            # Parse CSV data
            lines = r.text.split('\n')
            if not header:
                header = lines[0]
                all_data.append(header)
            data_lines = lines[1:]
            all_data.extend(data_lines)
            
            logger.info(f"Retrieved {len(data_lines)} data points for {month}")
            
            # Alpha Vantage has rate limits, so wait between requests
            # logger.info("Waiting 12 seconds before next request due to rate limits...")
            # time.sleep(12)  # Wait 12 seconds between requests
            
    else:
        logger.info(f"Using {interval} interval - fetching most recent data")
        # For other intervals, just fetch most recent data
        params = {
            'function': 'TIME_SERIES_INTRADAY',
            'symbol': ticker,
            'interval': interval,
            'apikey': api_key,
            'datatype': datatype,
            'adjusted': str(adjusted).lower(),
            'extended_hours': str(extended_hours).lower(),
            'outputsize': outputsize
        }
        
        url = 'https://www.alphavantage.co/query'
        r = requests.get(url, params=params)
        all_data = r.text.split('\n')
        logger.info(f"Retrieved {len(all_data)-1} data points")

    # Write combined data to file
    logger.info(f"Writing data to {data_file}")
    with open(data_file, "w") as f:
        f.write('\n'.join(all_data))

    logger.info("Data fetch completed successfully")
    return data_file

# # Example usage
for ticker in DJ_TITANS_50_TICKER:
    fetch_market_intraday(ticker, interval="1min", outputsize="full")
# fetch_market_intraday("AAPL", interval="1min", outputsize="full")


2025-02-17 12:10:33,408 - __main__ - INFO - Starting data fetch for MMM with 1min interval


2025-02-17 12:10:33,408 - INFO - Starting data fetch for MMM with 1min interval


2025-02-17 12:10:33,410 - __main__ - INFO - Using 1min interval - fetching data month by month for past year


2025-02-17 12:10:33,410 - INFO - Using 1min interval - fetching data month by month for past year


2025-02-17 12:10:33,411 - __main__ - INFO - Will fetch data for months: ['2025-02', '2025-01', '2024-12', '2024-11', '2024-10', '2024-09', '2024-08', '2024-07', '2024-06', '2024-05', '2024-04', '2024-03']


...

2025-02-17 12:30:46,072 - INFO - Retrieved 2 data points for 2024-03


2025-02-17 12:30:46,073 - __main__ - INFO - Writing data to /Users/benpfeffer/Desktop/CODE/volatility-strategy/db/cache/market_data/DJGT_1min.csv


2025-02-17 12:30:46,073 - INFO - Writing data to /Users/benpfeffer/Desktop/CODE/volatility-strategy/db/cache/market_data/DJGT_1min.csv


2025-02-17 12:30:46,074 - __main__ - INFO - Data fetch completed successfully


2025-02-17 12:30:46,074 - INFO - Data fetch completed successfully


In [13]:
def search_symbol(keywords: str, api_key: str = ALPHA_VANTAGE_API_KEY, datatype: str = "csv") -> dict:
    """
    Search for stock symbols and company information using Alpha Vantage's search endpoint.
    
    Args:
        keywords (str): Search keywords/terms
        api_key (str): Alpha Vantage API key
        datatype (str): Return data format - 'json' or 'csv'
    
    Returns:
        dict: Search results containing matching symbols and company information
    """
    base_url = "https://www.alphavantage.co/query"
    
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": keywords,
        "apikey": api_key,
        "datatype": datatype
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        if datatype == "csv":
            return response.text.split("\r\n")
        else:
            return response.text
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# Example usage
search_results = search_symbol("lvmh")
print(search_results)


['symbol,name,type,region,marketOpen,marketClose,timezone,currency,matchScore', 'LVMHF,Lvmh Moet Hennessy Vuitton SE,Equity,United States,09:30,16:00,UTC-04,USD,0.8889', 'LVMHX,LORD ABBETT CALIBRATED MID CAP VALUE FUND CLASS R6,Mutual Fund,United States,09:30,16:00,UTC-04,USD,0.8889', 'LVMUY,Lvmh Moet Hennessy Vuitton SE,Equity,United States,09:30,16:00,UTC-04,USD,0.6667', 'MOH.FRK,LVMH Moët Hennessy - Louis Vuitton Société Européenne,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.4444', 'MOH.DEX,LVMH Moët Hennessy - Louis Vuitton Société Européenne,Equity,XETRA,08:00,20:00,UTC+02,EUR,0.4000', 'MOHF.FRK,LVMH Moët Hennessy - Louis Vuitton Société Européenne,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.4000', 'MC.PAR,LVMH Moët Hennessy - Louis Vuitton Société Européenne,Equity,Paris,09:00,17:30,UTC+02,EUR,0.2222', '0HAU.LON,LVMH Moet Hennessy Louis Vuitton SE,Equity,United Kingdom,08:00,16:30,UTC+01,EUR,0.2051', '']


In [4]:
from src.config.paths import *
import requests
import datetime
from datetime import datetime, timedelta
import time
import csv
import json

def fetch_market_sentiment(tickers: str = None, 
                         topics: str = None,
                         time_from: str = None,
                         time_to: str = None,
                         sort: str = "LATEST",
                         limit: int = 1000,
                         api_key: str = ALPHA_VANTAGE_API_KEY,
                         output_dir: Path = NEWS_CACHE) -> dict:
    """
    Fetch market news and sentiment data from Alpha Vantage for at least one year.
    Rate limited to 75 requests per minute.
    
    Args:
        tickers (str, optional): Comma-separated stock/crypto/forex symbols to filter for
        topics (str, optional): Comma-separated news topics to filter for
        time_from (str, optional): Start time in YYYYMMDDTHHMM format
        time_to (str, optional): End time in YYYYMMDDTHHMM format  
        sort (str, optional): Sort order - 'LATEST', 'EARLIEST' or 'RELEVANCE'
        limit (int, optional): Number of results to return (max 1000)
        api_key (str): Alpha Vantage API key
        output_dir (Path): Directory to save output files
        
    Returns:
        dict: Market news and sentiment data
    """
    base_url = "https://www.alphavantage.co/query"
    
    # Calculate date one year ago
    if not time_from:
        one_year_ago = datetime.now() - timedelta(days=365)
        time_from = one_year_ago.strftime("%Y%m%dT%H%M")
    
    params = {
        "function": "NEWS_SENTIMENT",
        "apikey": api_key,
        "sort": sort,
        "limit": limit,
        "time_from": time_from
    }
    
    if tickers:
        params["tickers"] = tickers
    if topics:
        params["topics"] = topics
    if time_to:
        params["time_to"] = time_to
        
    try:
        # Add rate limiting delay to stay under 75 requests/min
        time.sleep(0.8)  # 800ms delay between requests
        
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Save to JSON file
        if tickers:
            output_file = output_dir / f"{tickers}_sentiment.json"
            with open(output_file, 'w') as f:
                json.dump(data, f, indent=4)
                
        return data
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching sentiment data: {e}")
        return None

for ticker in DJ_TITANS_50_TICKER:
    # Example usage
    sentiment_results = fetch_market_sentiment(tickers=ticker, 
                                        topics="technology,earnings,financial_markets,economy_macro,finance,economy_fiscal,economy_monetary,ipo,retail_wholesale",
                                        limit=1000)
    print(sentiment_results)


{'items': '0', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': []}
{'items': '0', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': []}
{'items': '0', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': []}
{'items': '0', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x 

In [10]:
import requests
import datetime
from datetime import datetime, timedelta
import time
import csv
import json

def fetch_federal_funds_rate(api_key, interval='daily', years=5, output_dir=FEDERAL_FUNDS_CACHE):
    """Fetch Federal Funds Rate data from Alpha Vantage.
    
    Args:
        api_key (str): Alpha Vantage API key
        interval (str, optional): Data interval - 'daily', 'weekly' or 'monthly'. Defaults to 'monthly'.
        years (int, optional): Number of years of historical data to fetch. Defaults to 5.
        
    Returns:
        dict: Federal Funds Rate data
    """
    base_url = "https://www.alphavantage.co/query"
    
    # Calculate date N years ago
    n_years_ago = datetime.now() - timedelta(days=365*years)
    time_from = n_years_ago.strftime("%Y-%m-%d")
    
    params = {
        "function": "FEDERAL_FUNDS_RATE",
        "apikey": api_key,
        "interval": interval,
        "outputsize": "full"
    }
    
    try:
        # Add rate limiting delay to stay under 75 requests/min
        time.sleep(0.8)  # 800ms delay between requests
        
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Filter data for last N years
        if data and 'data' in data:
            filtered_data = [d for d in data['data'] if d['date'] >= time_from]
            data['data'] = filtered_data
            
        # Save to CSV
        data_file = output_dir / f"federal_funds_{interval}.csv"
        with open(data_file, "w", newline='') as f:
            if data and 'data' in data:
                writer = csv.DictWriter(f, fieldnames=['date', 'value'])
                writer.writeheader()
                for row in data['data']:
                    writer.writerow(row)
            
        return data 
    except requests.exceptions.RequestException as e:
        print(f"Error fetching federal funds rate data: {e}")
        return None

# Example usage
fed_funds_rate = fetch_federal_funds_rate(ALPHA_VANTAGE_API_KEY, interval='daily', years=5)
if fed_funds_rate:
    print(json.dumps(fed_funds_rate, indent=2))


{
  "name": "Effective Federal Funds Rate",
  "interval": "daily",
  "unit": "percent",
  "data": [
    {
      "date": "2025-02-13",
      "value": "4.33"
    },
    {
      "date": "2025-02-12",
      "value": "4.33"
    },
    {
      "date": "2025-02-11",
      "value": "4.33"
    },
    {
      "date": "2025-02-10",
      "value": "4.33"
    },
    {
      "date": "2025-02-09",
      "value": "4.33"
    },
    {
      "date": "2025-02-08",
      "value": "4.33"
    },
    {
      "date": "2025-02-07",
      "value": "4.33"
    },
    {
      "date": "2025-02-06",
      "value": "4.33"
    },
    {
      "date": "2025-02-05",
      "value": "4.33"
    },
    {
      "date": "2025-02-04",
      "value": "4.33"
    },
    {
      "date": "2025-02-03",
      "value": "4.33"
    },
    {
      "date": "2025-02-02",
      "value": "4.33"
    },
    {
      "date": "2025-02-01",
      "value": "4.33"
    },
    {
      "date": "2025-01-31",
      "value": "4.33"
    },
    {
      "date"